In [1]:
#COMPUTE_SIFT.PY
# Sample solution code for computing SIFT feature pre-lab task

import cv2

class SiftDetector():
    def __init__(self, norm="L2", params=None):
        self.detector=self.get_detector(params)
        self.norm=norm

    def get_detector(self, params):
        if params is None:
            params={}
            params["n_features"]=0
            params["n_octave_layers"]=3
            params["contrast_threshold"]=0.03
            params["edge_threshold"]=10
            params["sigma"]=1.6

        detector = cv2.xfeatures2d.SIFT_create(
                nfeatures=params["n_features"],
                nOctaveLayers=params["n_octave_layers"],
                contrastThreshold=params["contrast_threshold"],
                edgeThreshold=params["edge_threshold"],
                sigma=params["sigma"])

        return detector


if __name__ == '__main__':
    # TASK 1

    # 1. Read image
    image = cv2.imread("image1.jpg")
    cv2.imshow("Colour picture", image)

    # 2. Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # 3. Initialize SIFT detector
    sift = SiftDetector()

    # 4. Detect SIFT features
    kp = sift.detector.detect(gray, None)

    # 5. Visualize detected features
    kp_gray = cv2.drawKeypoints(image, kp, gray)

    # Print number of SIFT features detected
    print("Number of SIFT features (default settings): ", len(kp))

    cv2.imshow("Keypoints image", kp_gray)
    cv2.imwrite('Keypoints image.jpg', kp_gray)
    cv2.waitKey(2000)
    cv2.destroyAllWindows()


Number of SIFT features (default settings):  1337


In [2]:
#TASK2.PY

import cv2

from compute_sift import SiftDetector


# Parameters for SIFT initializations such that we find only 25% of keypoints
params = {
    'n_features': 0,    #update this to 100, to view only 100 keypoints
    'n_octave_layers': 3,
    'contrast_threshold': 0.08,  # updated threshold.This value will vary for different images to view 25% of keypoints.
    'edge_threshold': 10,
    'sigma': 1.6
}


if __name__ == '__main__':
    # 1. Read image
    image = cv2.imread("image2.jpg")

    # 2. Convert image to grayscale
    grayImageOrig = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Initialize SIFT detector
    sift = SiftDetector(params=params)

    # Store SIFT keypoints of original image in a Numpy array
    kp,des1 = sift.detector.detectAndCompute(grayImageOrig, None)

    kp_gray = cv2.drawKeypoints(image, kp, grayImageOrig)
    cv2.imshow("Original keypoints", kp_gray)
    cv2.imwrite("Task_2_Original-keypoints.jpg", kp_gray)
    print("Number of SIFT features: ", len(kp))
    cv2.waitKey(2000)
    cv2.destroyAllWindows()

    #Upscale the image
    scale_percent = 110  # percent of original size
    width = int(image.shape[1] * scale_percent / 100)
    height = int(image.shape[0] * scale_percent / 100)
    dim = (width, height)
    # resize image
    resizedImage = cv2.resize(grayImageOrig, dim, interpolation=cv2.INTER_AREA)

    # Compute SIFT features for rescaled image
    kp_,des2 = sift.detector.detectAndCompute(resizedImage, None)
    kp_gray_ = cv2.drawKeypoints(resizedImage, kp_, resizedImage)
    cv2.imshow("Upscaled keypoints", kp_gray_)
    cv2.imwrite("Task_2_Upscaled-keypoints.jpg", kp_gray)
    cv2.waitKey(2000)
    cv2.destroyAllWindows()
    # BFMatcher with default params
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)
    print('matches:',len(matches))

    # Apply ratio test
    """This test rejects poor matches by computing the ratio between the best and second-best match. 
    If the ratio is below some threshold, the match is discarded as being low-quality."""
    good = []
    for m, n in matches:
        if m.distance < 0.80 * n.distance:
            good.append([m])

    # Draw matches
    result = cv2.drawMatchesKnn(
        grayImageOrig, kp,
        resizedImage, kp_,
        good, None, flags=2)

    cv2.imshow("Matched points", result)
    cv2.imwrite("Task_2_Matched-points.jpg", result)
    cv2.waitKey(2000)
    cv2.destroyAllWindows()

Number of SIFT features:  231
matches: 231


In [1]:
#TASK3.PY
# Sample solution for lab task 3 (SIFT robustness to changes in rotation)

import cv2
import math
import numpy as np

from compute_sift import SiftDetector

import sys
# Parameters for SIFT initializations such that we find only 25% of keypoints
params = {
    'n_features': 0,    #update this to 100, to view only 100 keypoints
    'n_octave_layers': 3,
    'contrast_threshold': 0.1,  # updated threshold.This value will vary for different images to view 25% of keypoints.
    'edge_threshold': 10,
    'sigma': 1.6
}

# Rotate an image
#
# image: image to rotate
# x:     x-coordinate of point we wish to rotate around
# y:     y-coordinate of point we wish to rotate around
# angle: degrees to rotate image by
#
# Returns a rotated copy of the original image
def rotate(image, x, y, angle):
    rot_matrix = cv2.getRotationMatrix2D((x, y), angle, 1.0)
    h, w = image.shape[:2]

    return cv2.warpAffine(image, rot_matrix, (w, h))


# Get coordinates of center point.
#
# image:  Image that will be rotated
# return: (x, y) coordinates of point at center of image
def get_img_center(image):
    height, width = image.shape[:2]
    center = height // 2, width // 2

    return center


if __name__ == '__main__':
    # Read image with OpenCV and convert to grayscale
    image = cv2.imread("Image1.jpg")
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Initialize SIFT detector
    sift = SiftDetector(params=params)

    # Store SIFT keypoints of original image in a Numpy array
    kp1, des1 = sift.detector.detectAndCompute(gray, None)
    OrigKeypoints = cv2.drawKeypoints(gray, kp1, None)
    print("Number of SIFT features: ", len(kp1))
    cv2.imshow("Original keypoints", OrigKeypoints)
    cv2.imwrite("Task_3_Original-keypoints.jpg", OrigKeypoints)
    cv2.waitKey(2000)
    cv2.destroyAllWindows()

    # center of image points. 'img_center' is in (Y, X) order.
    img_center = get_img_center(gray)
    x_coord = img_center[1]
    y_coord = img_center[0]

    # Degrees with which to rotate image
    angle = 90

    # Rotate image
    rotate_gray = rotate(gray, x_coord, y_coord, angle)

    # Compute SIFT features for rotated image
    kp2, des2 = sift.detector.detectAndCompute(rotate_gray, None)
    kp_gray = cv2.drawKeypoints(rotate_gray, kp2, None)

    # BFMatcher with default params
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)
    cv2.imshow("Rotated keypoints", kp_gray)
    cv2.imwrite("Task_3_Rotated-keypoints.jpg", kp_gray)
    cv2.waitKey(2000)
    cv2.destroyAllWindows()
    # Apply ratio test
    """This test rejects poor matches by computing the ratio between the best and second-best match. 
    If the ratio is below some threshold, the match is discarded as being low-quality."""
    good = []
    for m, n in matches:
        if m.distance < 0.80 * n.distance:
            good.append([m])

    # cv2.drawMatchesKnn
    result = cv2.drawMatchesKnn(
        gray, kp1,
        rotate_gray, kp2,
        good, None, flags=2)

    cv2.imshow("Matched points", result)
    cv2.imwrite("Task_3_Matched-points.jpg", result)
    cv2.waitKey(2000)
    cv2.destroyAllWindows()

Number of SIFT features:  132


In [ ]:
# add to plot to make look better
# plt.imshow(img, cmap=cm.gray, vmin=0, vmax=255)